In [ ]:
from datetime import datetime

import os
import requests
import time

from schema import NBAArena, NBATeam, WeatherSummary, NBAGame, NBAGameWeather, RegularSeason

In [ ]:
import csv
with open('../data/arena_locations.csv', mode ='r') as file:
    locations = [x for x in csv.DictReader(file)]

In [ ]:
def invert_locations(location_list: list) -> dict:
    
    arena_by_tricode = {}
    
    for arena in location_list:
        arena_by_tricode[arena.get('team_code')] = NBAArena(
            lat=arena.get('lat'),
            lon=arena.get('lon'),
            name=arena.get('arena'),
        )
        
    return arena_by_tricode

In [ ]:
import json
with open('../data/scheduleLeagueV2.json', 'r') as schedule_file:
    schedule_json = json.load(schedule_file)
    
games_2324 = schedule_json.get('leagueSchedule').get('gameDates')

In [ ]:
OPENING_NIGHT_2023 = "10/24/2023 00:00:00"
PLAYIN_START_2023 = "04/16/2024 00:00:00"

In [ ]:
opening_night_dt = datetime.strptime(OPENING_NIGHT_2023, "%m/%d/%Y %H:%M:%S")
playin_start_dt = datetime.strptime(PLAYIN_START_2023, "%m/%d/%Y %H:%M:%S")

In [ ]:
GAMEDATE_FORMAT = "%m/%d/%Y %H:%M:%S"

regular_season_games = []

for game_date in games_2324:
    game_dt = datetime.strptime(game_date.get("gameDate"), GAMEDATE_FORMAT)
    
    if (game_dt < playin_start_dt) and (game_dt >= opening_night_dt):
        if game_date.get('games')[0].get('gameLabel') in ['All-Star Game', 'Rising Stars Semifinal', 'Rising Stars Final']:
            pass
        else:
            regular_season_games.extend(game_date.get('games'))

In [ ]:
collect_team_games = {}
for game in regular_season_games:
    home_team = game.get('homeTeam').get('teamTricode')
    away_team = game.get('awayTeam').get('teamTricode')
    
    if (home_team in ['TAM', 'JAL', 'PAU', 'DLF', 'EST']) or (away_team in ['TAM', 'JAL', 'PAU', 'DLF']):
        print(game)
    
    
    for team in [home_team, away_team]:
        if team in collect_team_games.keys():
            collect_team_games[team] += 1
        else:
            collect_team_games[team] = 1
            
assert len(regular_season_games) == (41*30) + 1, "Game count should match"

In [ ]:
# with open('regular_season_2023_2024.json', 'w') as regular_season_json:
#     json.dump(regular_season_games, regular_season_json)

In [ ]:
arena_by_tricode = invert_locations(locations)

In [ ]:
special_games = {
    "0022301229": "LAS",
    "0022301230": "LAS",
    "0062300001": "LAS",
    "0022300527": "PAR",
    "0022300965": "AUS",
    "0022300981": "AUS",
}

In [ ]:
def type_game(game_dict: dict) -> NBAGame:
    """Convert a game dict into a 'typed' game object."""
    
    home_tricode = game_dict.get("homeTeam").get("teamTricode")
    
    home_team = NBATeam(
        teamTricode=home_tricode,
    )

    arena = arena_by_tricode.get(home_tricode)

    if arena.name != game_dict.get('arenaName'):
        if game_dict.get('gameId') in special_games:
            arena = arena_by_tricode.get(special_games.get(game_dict.get('gameId')))
        elif home_team.teamTricode == "ORL":
            pass

    return NBAGame(
        game_id=game_dict.get("gameId"),
        game_code=game_dict.get("gameCode"),
        home_team=home_team,
        venue=arena,
        home_datetime=game_dict.get("homeTeamTime"),
        weather_summary=None,
    )

In [ ]:
typed_regular_season = [type_game(x) for x in regular_season_games]

In [ ]:
def call_openweathermap(lat: str, lon: str, date: str):
    """Call OpenWeatherMap API."""
    
    api_key = os.environ.get("OPENWEATHERMAP_KEY")
    url = f"https://api.openweathermap.org/data/3.0/onecall/day_summary?lat={lat}&lon={lon}&date={date}&appid={api_key}&units=imperial"
    
    req = requests.get(url)
    
    weather_json = req.json()
    
    return weather_json


def get_weather_for_game(game: NBAGame) -> NBAGameWeather:
    """Call weather API to get appropriate weather summary for an NBAGame."""
    
    date_string = game.home_datetime[0:10]
    
    weather_json = call_openweathermap(
        lat = str(game.venue.lat),
        lon = str(game.venue.lon),
        date = date_string
    )
    
    game_weather = WeatherSummary(
        temp_high_f = weather_json.get('temperature').get('max'),
        temp_low_f = weather_json.get('temperature').get('min'),
        precipitation_mm = weather_json.get('precipitation').get('total'),
        wind_max_mph = weather_json.get('wind').get('max').get('speed'),
    )
    
    return game_weather

In [ ]:
games_with_weather = []

for game in typed_regular_season:
    weather = get_weather_for_game(game)
    
    games_with_weather.append(
        NBAGameWeather(
            game_id=game.game_id,
            game_code=game.game_code,
            home_team=game.home_team,
            venue=game.venue,
            home_datetime=game.home_datetime,
            weather_summary=weather,
        )
    )
    
    if len(games_with_weather) % 100 == 0:
        print(len(games_with_weather), end=" | ")
    
    # time.sleep(0.1) # try not to get throttled

In [ ]:
# with open('../outputs/save_weather_2023_2024.json', 'w') as sw:
#     json.dump([x.model_dump_json() for x in games_with_weather], sw)